In [1]:
cd ../src

c:\Users\jeong\Desktop\python\action-recognition-practice\src


In [2]:
import os
import numpy as np
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from pytorch_lightning import Trainer
from pytorch_lightning.metrics.functional import accuracy
from typing import Any, Callable, Dict, List, Optional, Tuple, cast
import matplotlib.pyplot as plt
import datasets

plt.rcParams['figure.figsize'] = [8, 6]

In [3]:
root_path = "../data/hmdb/processed"
npz_filename = os.path.abspath(os.path.join(root_path, "subset.npz"))
npz_filename

'c:\\Users\\jeong\\Desktop\\python\\action-recognition-practice\\data\\hmdb\\processed\\subset.npz'

In [4]:
trainer = pl.Trainer(gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [5]:
dm = datasets.nia2019v1.NIA2019V1DataModule(npz_filename, batch_size=64)

In [6]:
class PracticeModel(pl.LightningModule):
    """
    (batch, channel, time, height, width)
    """
    def __init__(self, num_classes: int = 25):
        super().__init__()
        self.num_classes = num_classes
        self.model = models.video.r2plus1d_18(True)
        for param in self.model.parameters():
            param.requires_grad = False
        #self.model.eval()
        self.model.fc = nn.Linear(self.model.fc.in_features, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.model(x)
        x = self.fc2(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(), lr=0.01)
        sched = torch.optim.lr_scheduler.CosineAnnealingLR(opt, self.num_classes)
        return [opt], [sched]

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y, num_classes=self.num_classes)
        
        result = pl.TrainResult(minimize=loss)
        result.log('train_loss', loss, prog_bar=True)
        result.log('train_acc', acc, prog_bar=True)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)

        loss = F.cross_entropy(y_hat, y)
        acc = accuracy(y_hat, y, num_classes=self.num_classes)
        
        result = pl.TrainResult(minimize=loss)
        result.log('val_loss', loss, prog_bar=True)
        result.log('val_acc', acc, prog_bar=True)
        return result


In [7]:
model = PracticeModel(num_classes=25)

In [16]:
trainer.fit(model, dm)

KeyboardInterrupt: 